In [ ]:
from ultralytics import YOLO
import os


DATA_YAML   = "/home/carbucks/Carbucks_YOLO/dataset.yaml"
PROJECT_DIR = "/home/carbucks/runs/detect"       
RUN_NAME    = "y11x_run"                  
PRETRAINED  = "yolo11x.pt"                       
TEST_IMAGES = "/home/carbucks/Carbucks_YOLO/images/test"   


os.makedirs(PROJECT_DIR, exist_ok=True)


: 

In [ ]:
# Init YOLO11x model (will download if not present in cache)
model = YOLO(PRETRAINED)


In [ ]:
# Train with full hyperparameters (edit as needed)
results = model.train(
    data=DATA_YAML,
    project=PROJECT_DIR,
    name=RUN_NAME,
    epochs=1,
    imgsz=1280,
    batch=-1,                 # auto-batch to fit VRAM
    device=0,
    workers=8,
    seed=0,
    deterministic=True,
    optimizer="auto",
    pretrained=True,
    save=True,
    save_period=10,
    cache=False,
    val=True,
    amp=True,
    close_mosaic=10,
    rect=False,
    multi_scale=False,
    cos_lr=False,
    # ---- loss/solver knobs
    box=7.5,
    cls=0.5,
    dfl=1.5,
    iou=0.2,
    label_smoothing=0.0,
    nbs=64,
    overlap_mask=True,
    mask_ratio=4.0,
    dropout=0.0,
    # ---- augmentation (photometric & geometric)
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0,
    flipud=0.0, fliplr=0.5, bgr=0.0,
    mosaic=1.0, mixup=0.1, cutmix=0.0, copy_paste=0.0, copy_paste_mode="flip",
    erasing=0.0,
)

# Resolve run paths
run_dir   = os.path.join(PROJECT_DIR, RUN_NAME)
weights_d = os.path.join(run_dir, "weights")
best_pt   = os.path.join(weights_d, "best.pt")
last_pt   = os.path.join(weights_d, "last.pt")

print("\n=== TRAIN OUTPUTS ===")
print(f"Run directory : {run_dir}")
print(f"Best weights  : {best_pt}")
print(f"Last weights  : {last_pt}")


In [ ]:
# Reload best model checkpoint
model = YOLO(best_pt)

# Run validation
val_res = model.val(
    data=DATA_YAML,
    project=PROJECT_DIR,
    name=f"{RUN_NAME}_val",
    imgsz=1280,
    batch=-1,
    device=0,
    save_json=True,
    plots=True,
    augment=True,   # TTA-style eval
)

val_dir = os.path.join(PROJECT_DIR, f"{RUN_NAME}_val")
print("\n=== VAL OUTPUTS ===")
print(f"Val directory : {val_dir}")
print(f"mAP50-95      : {val_res.results_dict.get('metrics/mAP50-95(B)', 'n/a')}")


In [ ]:
if os.path.isdir(TEST_IMAGES):
    pred_res = model.predict(
        source=TEST_IMAGES,
        project=PROJECT_DIR,
        name=f"{RUN_NAME}_pred",
        imgsz=1280,
        conf=0.25,
        iou=0.5,
        device=0,
        save=True,       # saves annotated images
        save_txt=False,
        save_conf=True,
    )
    pred_dir = os.path.join(PROJECT_DIR, f"{RUN_NAME}_pred")
    print("\n=== PREDICTION OUTPUTS ===")
    print(f"Pred directory: {pred_dir}")
else:
    print("\n(No TEST_IMAGES folder found; skipping predict.)")
